In [41]:
import os
import pandas as pd
from PIL import Image
Image.MAX_IMAGE_PIXELS = 999999999

In [42]:
src = r'\\10.162.80.6\Kyu_Sync\server for undergrads\Stephen'

csvs = [_ for _ in os.listdir(src) if _.endswith('csv')]
fns = [os.path.splitext(_)[0] for _ in csvs]
trues = [_.replace('parameters','tile') for _ in fns]
falses = [_.replace('tile','false_positive') for _ in trues]

In [43]:
for csv,true,false in zip(csvs,trues,falses):
    csvdf = pd.read_csv(os.path.join(src,csv))
    truepngs = [os.path.splitext(_)[0][5:] for _ in os.listdir(os.path.join(src,true,'image')) if _.endswith('png')]
    falsepngs = [os.path.splitext(_)[0][5:] for _ in os.listdir(os.path.join(src,false,'image')) if _.endswith('png')]
    allpngs = truepngs+falsepngs
    pngs2move = [_ for _ in list(csvdf.index) if str(_) not in allpngs]
    pngs2movedf = csvdf.iloc[pngs2move]

    img_name = csv.replace('parameters','tissue_region').replace('csv','tif')
    img_pth = os.path.join(src,img_name)
    img = Image.open(img_pth)
    bw_img = Image.open(img_pth.replace('tissue_region','filtered'))

    for idx, row in pngs2movedf.iterrows():
        x = row['x']
        y = row['y']
        (left, upper, right, lower) = (x - 50, y - 50, x + 50, y + 50)
        im_crop = img.crop((left, upper, right, lower))
        bw_crop = bw_img.crop((left, upper, right, lower))
        im_crop.save(os.path.join(*[src,true,'image','tile_{:d}.png'.format(idx)]))
        bw_crop.save(os.path.join(*[src,true,'mask','tile_{:d}.png'.format(idx)]))